In [13]:
import pandas as pd
import numpy as np

# pd.merge()

# 예제 데이터 프레임 생성
df1 = pd.DataFrame({'key' : ['A', 'B', 'C'], 'value': [1,2,3]})
df2 = pd.DataFrame({'key' : ['A', 'B', 'D'], 'value': [4,5,6]})

# 두 데이터 프레임 병합
merged_df = pd.merge(df1, df2, on = 'key', how = 'inner')
print(merged_df)

merged_df_outer = pd.merge(df1, df2, on='key', how = 'outer')
print(merged_df_outer)

# pd.melt()
data = {
    'Date' : ['2024-07-01', '2024-07-02', '2024-07-03', '2024-07-03'],
    'Temperature' : [10,20,25,20],
    'Humidity' : [60,65,70,21]
}
df = pd.DataFrame(data)
print(df)

df_melted = pd.melt(df,
                    id_vars=['Date'],
                    var_name='Variable',
                    value_name='Value',
                    value_vars=['Temperature', 'Humidity'])   # value_vars가 없어도 모두 출력이 된다.
print(df_melted)

# pivot_table()
df_pivot_table = df_melted.pivot_table(index = 'Date',
                                       columns = 'Variable',
                                       values = 'Value').reset_index()
print(df_pivot_table)

# rename()
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/dat.csv')
df = df.rename(columns={'Dalc':'dalc', 'Walc':'walc'})      # columns = {}에서 columns를 반드시 써주기
print(df.columns)

  key  value_x  value_y
0   A        1        4
1   B        2        5
  key  value_x  value_y
0   A      1.0      4.0
1   B      2.0      5.0
2   C      3.0      NaN
3   D      NaN      6.0
         Date  Temperature  Humidity
0  2024-07-01           10        60
1  2024-07-02           20        65
2  2024-07-03           25        70
3  2024-07-03           20        21
         Date     Variable  Value
0  2024-07-01  Temperature     10
1  2024-07-02  Temperature     20
2  2024-07-03  Temperature     25
3  2024-07-03  Temperature     20
4  2024-07-01     Humidity     60
5  2024-07-02     Humidity     65
6  2024-07-03     Humidity     70
7  2024-07-03     Humidity     21
Variable        Date  Humidity  Temperature
0         2024-07-01      60.0         10.0
1         2024-07-02      65.0         20.0
2         2024-07-03      45.5         22.5
Index(['school', 'sex', 'paid', 'famrel', 'freetime', 'goout', 'dalc', 'walc',
       'health', 'absences', 'grade'],
      dtype='object')


In [36]:
# 기출문제 제9회(2024년 11월 30일 시행)
#1번(groupby, pivot_table 활용 사례)
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_1.csv')
print(df.head())

df['총대출액'] = df['금액1'] + df['금액2']

dfgender = df.groupby(['year', 'gender', '지역코드'])['총대출액'].sum().reset_index()
print(dfgender)
# 각 연도 및 성별에 따른 차이를 계산하기 위해 피벗 테이블을 생성합니다.
df_pivot = dfgender.pivot_table(index = ['year', '지역코드'], columns = 'gender', values = '총대출액', fill_value=0)
print(df_pivot)

#2번(rename, melt, merge 활용 사례)
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_2.csv')
print(df.head())
df1 = df[df['구분'] == '검거건수'].set_index('연도').drop(columns='구분')
df2 = df[df['구분'] == '발생건수'].set_index('연도').drop(columns='구분')
dfratio = df1/df2 #검거율
dfmax = dfratio.idxmax(axis=1)
print(dfmax)
dfmax_reset = dfmax.reset_index().rename({0:'범죄유형'}, axis=1)
print(dfmax_reset)

#기존 df를 긴 유형의 데이터프레임으로 바꾸기(pd.melt() 활용)
#각 연도별 범죄유형에 해당하는 검거건수의 총합을 출력합니다.
df1_melt = df1.reset_index().melt(id_vars = '연도', var_name='범죄유형', value_name = '검거건수')
print(df1_melt)
dffinal = pd.merge(dfmax_reset, df1_melt, on = ['연도', '범죄유형'], how='left')
print(dffinal)

#3번(apply, rename, merge, np.where를 활용한 결측치 대치)
df=pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_3.csv')
print(df.head())

print(df.isna().sum())
df['평균만족도'] = df['평균만족도'].fillna(df['평균만족도'].mean())

#부서와 등급에 따라 평균 근속연수로 대치합니다.
mean_tenure = df.groupby(['부서', '등급'])['근속연수'].mean().apply(np.floor).reset_index().rename(columns={'근속연수':'평균근속연수'})
print(mean_tenure)

#merge로 결측치 대치
df = df.merge(mean_tenure, on=['부서', '등급'], how = 'left')
df['근속연수'] = np.where(
    df['근속연수'].isnull(),
    df['평균근속연수'],
    df['근속연수']
)
df.drop(columns = '평균근속연수', inplace=True)
print(df.isna().sum())

   year  gender  지역코드    금액1    금액2
0  2018       0   143  70713  43210
1  2019       0   183  40306  39663
2  2017       0   129  26646  41631
3  2019       0   161  56843  12455
4  2019       0   174  26371   9014
    year  gender  지역코드    총대출액
0   2015       0   131   37881
1   2015       0   151   32197
2   2015       0   195  161121
3   2015       0   197   93317
4   2015       1   101   87088
..   ...     ...   ...     ...
89  2020       1   102  105908
90  2020       1   168   85313
91  2020       1   172   97854
92  2020       1   192  109436
93  2020       1   195   34792

[94 rows x 4 columns]
gender           0         1
year 지역코드                   
2015 101       0.0   87088.0
     118       0.0  100609.0
     130       0.0   41599.0
     131   37881.0       0.0
     150       0.0   56775.0
...            ...       ...
2020 176   85767.0       0.0
     178   71066.0       0.0
     192       0.0  109436.0
     195       0.0   34792.0
     196   76900.0       0.0

[91 rows x 